# Docker setup for nginx servers

## NGINX config file
Used for setting up server

Write nginx.conf config to setup server and link appropriate Docker containers

In [5]:
%%writefile nginx.conf

# Define the server block for Park manager aspect of app
server {
    # Listen on port 80 (HTTP)
    listen 80;
    
    # Set the server name (can be any name or IP) - NOTE: this host name should be in host field in request
    server_name parkman.localhost;
    
    # Define the location block for handling requests
    location / {
        # Proxy all requests to the Flask app running in the 'manager_app' service on port 80
        proxy_pass http://manager_app:80;
        
        # Preserve the original Host header from the client
        proxy_set_header Host $host;
        
        # Pass the real client's IP address to the Flask app
        proxy_set_header X-Real-IP $remote_addr;

        # Pass additional headers for proper request handling
        proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
        proxy_set_header X-Forwarded-Proto $scheme;
    }



    location /user { #if locaton /user
         # Proxy all requests to the Flask app running in the 'mobile_app' service on port 80
         proxy_pass http://mobile_app:80;
        
#         # Preserve the original Host header from the client
         proxy_set_header Host $host;
        
#         # Pass the real client's IP address to the Flask app
         proxy_set_header X-Real-IP $remote_addr;

#         # Pass additional headers for proper request handling
         proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
         proxy_set_header X-Forwarded-Proto $scheme;
    }

}

# Define the server block for the MobileAPP aspect of app, TODO: implement after MobileAPP is implemented
# server {
#     # Listen on port 80 (HTTP)
#     listen 80;
    
#     # Set the server name (can be any name or IP)
#     server_name MobileAPP;
    
#     # Define the location block for handling requests
#     location / {
#         # Proxy all requests to the Flask app running in the 'mobile_app' service on port 80
#         proxy_pass http://mobile_app:80;
        
#         # Preserve the original Host header from the client
#         proxy_set_header Host $host;
        
#         # Pass the real client's IP address to the Flask app
#         proxy_set_header X-Real-IP $remote_addr;

#         # Pass additional headers for proper request handling
#         proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
#         proxy_set_header X-Forwarded-Proto $scheme;
#     }
# }


Overwriting nginx.conf


Create Docker file for nginx container setup

In [5]:
%%writefile nginx.dockerfile

# Use the official Nginx image as the base
FROM nginx:latest

# Remove the default Nginx configuration file from container
RUN rm /etc/nginx/conf.d/default.conf

# Copy our custom Nginx configuration into the container
COPY nginx.conf /etc/nginx/conf.d

# Expose port 80 to allow external access to the container's port 80
EXPOSE 80

#run command on container create which starts nginx. nginx works as a foreground process since docker considers daemon processes as "not running", hence it shutsdown the container
CMD ["nginx", "-g", "daemon off;"]

Overwriting nginx.dockerfile


## Embedding Nginx service into docker compose

In [6]:
from ruamel.yaml import YAML

# Create a YAML object
yaml = YAML()
yaml.preserve_quotes = True  # Preserves quotes in the YAML file
yaml.indent(mapping = 2, sequence = 2, offset = 2)

#Setup file edit path 
docker_compose_path = '../docker-compose.yml'  

# Read the docker-compose.yml file
with open(docker_compose_path, 'r') as file:
    docker_compose = yaml.load(file)

# Define Nginx service configuration
nginx_service = {
    'build': {
        'context': './nginx',
        'dockerfile': 'nginx.dockerfile'
    },
    'depends_on': ['manager_app'], #TODO: add service 'mobile_app' after implemented
    'networks': ['app-network'],
    'ports': ['80:80'],
    'labels': [
        'traefik.enable=true',
        'traefik.http.routers.nginx.rule= Host(`parkman.localhost`)',
        'traefik.http.routers.nginx.entrypoints=web',
        'traefik.http.services.nginx.loadbalancer.server.port=80'
    ]
    
}



docker_compose['services']['nginx']=nginx_service


# Write the updated configuration back to docker-compose.yml
with open(docker_compose_path, 'w') as file:
    yaml.dump(docker_compose, file)

print("\ndocker-compose.yml has been updated successfully.")



docker-compose.yml has been updated successfully.
